# Яндекс Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные Яндекс Музыки о поведении пользователей находятся в файле `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Необходимо проверить данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки необходимо найти возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в четыре этапа:
 1. [Обзор данных](#section_1).
 2. [Предобработка данных](#section_2).
 3. [Проверка гипотез](#section_3).
 4. [Итоги исследования](#section_4).



<a id='section_1'></a>
## Обзор данных





Импорт основного инструмента — библиотеки `pandas`

In [ ]:
import pandas as pd

Чтение файл `yandex_music_project.csv` из папки `/datasets` и сохранение его в переменной `df`:

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Вывод на экран первые десять строк таблицы:

In [ ]:
print(df.head(10))

    user_id                        track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой           unknown  ruspop   
9  E772D5C0                    Pessimist           unknown   dance   

               city      time        day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

Описание данных (согласно документации):
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Получение общей информацию о таблице c помощью метода `info()`

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Выводы:**

В таблице семь столбцов. Тип данных во всех столбцах — `object`. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В названиях колонок видны **нарушения стиля**:
* Строчные буквы сочетаются с прописными.
* Встречаются пробелы.
* Название колонки `userID` следует переписать в "_змеином_регистре_".

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются **пропуски** в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

<a id='section_2'></a>
## Предобработка данных

Необходимо исправить стиль в заголовках столбцов, исключить пропуски. Затем проверите данные на дубликаты.

### Стиль заголовков

Исходные названия столбцов:

In [ ]:
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Хорошим стилем считается:
* слова в названии записаны в « _змеином_регистре_ »,
* все символы являются строчными,
* устранены пробелы.

Для этого будут переименованы колонки следующим образом:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [ ]:
df = df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

Проверьтка результат. Для этого ещё раз необходимо вывести на экран названия столбцов:

In [ ]:
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Пропуски значений

Для начала необходимо посчитать, сколько в таблице пропущенных значений. Для этого достаточно пары методов `pandas`:

In [ ]:
print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создайте список `columns_to_replace`, переберая его элементы циклом `for` и для каждого столбца выполните замену пропущенных значений:

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']
for col in columns_to_replace: # перебор названий столбцов в цикле и
    df[col] = df[col].fillna('unknown') # замена пропущенных значений на 'unknown'

Проверка, что в таблице не осталось пропусков.

In [ ]:
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Дубликаты

Посчёт явных дубликатов в таблице:

In [ ]:
print(df.duplicated().sum())

3826


Вызовом специальным методом из `pandas`, удалю явные дубликаты:

In [ ]:
df = df.drop_duplicates()

Ещё раз посчитаю явные дубликаты в таблице — убедиться, что полностью от них избавился:

In [ ]:
print(df.duplicated().sum())

0


Теперь необходимо избавиться от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведу на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
1. извлеку нужный столбец датафрейма; 
2. применю к нему метод сортировки;
3. для отсортированного столбца вызову метод, который вернёт уникальные значения из столбца.

In [ ]:
df_genre = df['genre']
df_genre = df_genre.sort_values()
print(df_genre.unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Просмотрев список, нашел неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

К примеру следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу использую метод `replace()` с двумя аргументами: списком строк-дубликатов (включащий *hip*, *hop* и *hip-hop*) и строкой с правильным значением. Нужно исправить колонку `genre` в таблице `df`: заменив каждое значение из списка дубликатов на верное. Вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [ ]:
df['genre'] = df['genre'].replace(['hip','hop','hip-hop'],'hiphop')

Проверька, что заменены неправильные названия. Вывод отсортированного списока уникальных значений столбца `genre`:

In [ ]:
df_genre = df['genre']
df_genre = df_genre.sort_values()
print(df_genre.unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы:**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Были исправлены заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения были замены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

<a id='section_3'></a>
## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверю это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделю пользователей Москвы и Санкт-Петербурга.
* Сравню, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполните каждый из расчётов по отдельности. 

Оценю активность пользователей в каждом городе, сгруппировав данные по городу и посчитав прослушивание в каждой группе.



In [ ]:
df_city = df.groupby('city')['user_id'].count()
print(df_city)

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппирую данные по дню недели и посчитаю прослушивания в понедельник, среду и пятницу, учитывая, что в данных есть информация о прослушиваниях только за эти дни.


In [ ]:
df_days = df.groupby('day')['user_id'].count() # Подсчёт прослушиваний в каждый из трёх дней
print(df_days)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64


В среднем, пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создам функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

Внутри функция сохраняет в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применю последовательную фильтрацию с логической индексацией (или сложные логические выражения в одну строку).

Затем посчитаю значения в столбце `user_id` получившейся таблицы. Результат сохраню в новую переменную. Верну эту переменную из функции.

In [ ]:
def number_tracks(day, city): # Объявляется функция с двумя параметрами: day, city.
    track_list = df # В переменной track_list сохраняются те строки таблицы df, для которых 
    track_list = track_list[track_list['day']==day] # значение в столбце 'day' равно параметру day и одновременно значение
    track_list = track_list[track_list['city']==city] # в столбце 'city' равно параметру city (используйте последовательную фильтрацию
    track_list_count = track_list['user_id'].count() # В переменной track_list_count сохраняется число значений столбца 'user_id',
    return track_list_count # Функция возвращает число - значение track_list_count.

Вызову `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [ ]:
number_tracks('Monday', 'Moscow')# количество прослушиваний в Москве по понедельникам

15740

In [ ]:
number_tracks('Monday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [ ]:
number_tracks('Wednesday', 'Moscow')# количество прослушиваний в Москве по средам

11056

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

7003

In [ ]:
number_tracks('Friday', 'Moscow')# количество прослушиваний в Москве по пятницам

15945

In [ ]:
number_tracks('Friday', 'Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

5895

Создам c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые получил с помощью `number_tracks`.

In [ ]:
data = []
columns = ['city', 'monday', 'wednesday', 'friday']

for city in ['Moscow', 'Saint-Petersburg']:
    row_data = []
    row_data.append(city)
    for day in ['Monday', 'Wednesday', 'Friday']:
        row_data.append(number_tracks(day, city))
    data.append(row_data)

info = pd.DataFrame(data=data, columns=columns)# Таблица с результатами
#display(info)

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраню таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [ ]:
moscow_general = df[df['city']=='Moscow']
spb_general    = df[df['city']=='Saint-Petersburg'] 

Создам функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [ ]:
def genre_weekday(table, day, time1, time2):
    # последовательная фильтрация
    genre_df = table[table['day']==day] # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = genre_df[genre_df['time']<time2] # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time']>time1] # оставляем в genre_df только те строки genre_df, у которых время больше time1
    
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    
    return genre_df_sorted[:10] # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [ ]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [ ]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [ ]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [ ]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, то:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппирую таблицу `moscow_general` по жанру и посчитаю прослушивания треков каждого жанра методом `count()`. Затем отсортирую результат в порядке убывания и сохраню его в таблице `moscow_genres`.

In [ ]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

print(moscow_genres.head(10)) # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


Теперь повторю то же и для Петербурга.


In [ ]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

print(spb_genres.head(10)) # просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


<a id='section_4'></a>
## Итоги исследования

Были проверены три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. (Первая гипотеза полностью подтвердилась.)

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

(Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.)

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

(Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.)